# Blending models - example: Steel production


We want to produce a predefined quantity of a product, e.g. steel.
The produced product must have certain characteristics (grades) - see section below. 
In stock we have different possible ingredients, e.g. ores, and these  ingredients contribute differently to the requirements and have different availablility and prices - see section below.

We want to determine the recipie which minimizes the production costs. 

source: @gueret1999applications p.62

In [ ]:
import pandas as pd
import pyomo.environ as pyo

# Example data

For our example we consider the following characteristics of the final product and the following raw materials with the the following properties:

In [ ]:
requirements = pd.DataFrame(
    {'element' : ['C','Cu','Mn'],
    'min_grade': [2.,0.4,1.2],
    'max_grade': [3,0.6, 1.65]}
)

raw_mat = pd.DataFrame({
    'raw_mat': ['Fe1','Fe2','Fe3','Cu1','Cu2','Al1','Al2'],
    'C': [2.5,3.,0.,0.,0.,0.,0],
    'Cu':[0.,0.,0.3,90.,96.,0.4,0.6],
    'Mn':[1.3,0.8,0.,0.,4.,1.2,0.],
    'availability':[400.,300.,600.,500.,200.,300.,250.],
    'costs':[200.,250.,150.,220.,240.,200.,165.],
})

demand = 500.

print('requirements')
display(requirements)
print('raw materials')
display(raw_mat)
print('Demand of the final product: ' + str(demand))

requirements


,element,min_grade,max_grade
0,C,2.0,3.00
1,Cu,0.4,0.60
2,Mn,1.2,1.65


raw materials


,raw_mat,C,Cu,Mn,availability,costs
0,Fe1,2.5,0.0,1.3,400.0,200.0
1,Fe2,3.0,0.0,0.8,300.0,250.0
2,Fe3,0.0,0.3,0.0,600.0,150.0
3,Cu1,0.0,90.0,0.0,500.0,220.0
4,Cu2,0.0,96.0,4.0,200.0,240.0
5,Al1,0.0,0.4,1.2,300.0,200.0
6,Al2,0.0,0.6,0.0,250.0,165.0


Demand of the final product: 500.0


# Model 

Before we give an algebraic representation of the model, we introduce the relevant problem notions:

## sets

- $raw$ - set of raw materials
- $comp$ - set of grade requirements

## variables

- $use_r$ - used quantity in optimal recipie of raw material $r$

## parameter

- $demand$ - demand of the product
- $produce$ - produced quatity of the product
- $cost_r$ - costs of raw material $r$
- $P_{rc}$ - percentage of component $c$ in raw material $r$
- $Pmin_c$ - minimal requirement of component $c$ in product
- $Pmax_c$ - maximal requirement of component $c$ in product


## objective

- find the recipe with minimal costs fulfilling the requirements

## constraints

- (c1) produced quatity is given by the sum of the used rat materials (in tons)
- (c2) lower limit on the components in the final product
- (c3) upper limit on the components in the final product
- (c4) use only available quatities of the raw materials
- (c5) produced quatity fulfills demands


## deterministic model

$$
\begin{array}{llll}
\min & \sum_{r\in raw} cost_r \cdot use_r & & \\
s.t. & produce = \sum_{r\in raw} use_r & & (c1)\\
     & \sum_{r\in raw} P_{rc}\cdot use_r \geq Pmin_c \cdot produce & , \forall c\in comp & (c2)\\
     & \sum_{r\in raw} P_{rc}\cdot use_r \leq Pmax_c \cdot produce &  , \forall c\in comp & (c3)\\
     & use_r \leq avail_r &  , \forall r\in raw & (c4) \\
     & produce \geq demand & & (c5) \\
     & use_r,produce \geq 0 & & (c6)
\end{array}
$$

In [ ]:
def alloy_blending(requirements,raw_mat, demand):
    m = pyo.ConcreteModel('alloy production')
    
    # sets
    m.raw = pyo.Set(initialize = raw_mat.raw_mat)
    m.comp = pyo.Set(initialize = requirements.element)
    
    # parameter
    @m.Param(m.raw, doc = 'costs of raw mat r')
    def costs(m,r):
        return raw_mat.loc[raw_mat['raw_mat'] == r, 'costs'].values[0]
    @m.Param(m.raw, m.comp, doc = 'percentage of component c in raw material r')
    def P(m,r,c):
        return raw_mat.loc[raw_mat['raw_mat'] == r, c].values[0]
    @m.Param(m.comp, doc= 'minimal percentage of component c in final product')
    def Pmin(m,c):
        return requirements.loc[requirements['element'] == c, 'min_grade'].values[0]
    @m.Param(m.comp, doc= 'maximal percentage of component c in final product')
    def Pmax(m,c):
        return requirements.loc[requirements['element'] == c, 'max_grade'].values[0]
    @m.Param(m.raw, doc = 'availability of raw materials')
    def avail(m,r):
        return raw_mat.loc[raw_mat['raw_mat'] == r, 'availability'].values[0]
    @m.Param(doc = 'product demand')
    def demand(m):
        return demand
    
    # variables
    m.use = pyo.Var(m.raw, domain = pyo.NonNegativeReals, doc = 'used tons of raw material r')
    m.produced = pyo.Var(domain = pyo.NonNegativeReals, doc = 'tons produced')
    
    # objective
    def objective(m):
        return pyo.summation(m.costs, m.use)
    m.OBJ = pyo.Objective(rule = objective(m), sense = pyo.minimize)
    
    # constraints
    @m.Constraint(doc ='tons produced')
    def c1(m):
        return m.produced == pyo.summation(m.use)
    
    @m.Constraint(m.comp, doc = 'lower limit of grade in final product')
    def c2(m, c):
        return sum(m.P[r,c] * m.use[r] for r in m.raw) >= m.Pmin[c] * m.produced
    @m.Constraint(m.comp, doc = 'upper limit of grade in final product')
    def c3(m, c):
        return sum(m.P[r,c] * m.use[r] for r in m.raw) <= m.Pmax[c] * m.produced
    @m.Constraint(m.raw, doc = 'use component c maximal w.r.t. availability of c')
    def c4(m, r):
        return m.use[r] <= m.avail[r]
    @m.Constraint(doc = 'produce at least the demand')
    def c5(m):
        return m.produced >= m.demand

    
    # solver
    solver = pyo.SolverFactory('glpk')
    solver.solve(m)
    
    return m

# helper functions

In [ ]:
def extract_solution(m):
    solution = {r: [pyo.value(m.use[r])] for r in m.raw}
    output = pd.DataFrame(data = solution)
    output.index = ['used']
    return output

def visualize_solution(solution):    
    output = solution.T
    output['raw_mat'] = output.index
    output = pd.merge(output, raw_mat)
    for i in requirements.element:
        output[i] = output[i] * output['used'] / sum(output['used'])
    costs = round(sum(output['used'] * output['costs']),2)
    print('production costs: ' + str(costs))
    produced = round(sum(output['used']),2)
    print('tons produced: ' + str(produced))
    remove_col = set(output.columns) - {'C', 'Cu', 'Mn'}
    output = pd.DataFrame(output.drop(columns = remove_col).sum(axis = 0))
    output.rename(columns={0: 'grade in final product'} , inplace=True)
    return output

# inspect soulution

In [ ]:
m = alloy_blending(requirements,raw_mat, demand)
sol = extract_solution(m)
display(sol)

visualize_solution(sol)

,Fe1,Fe2,Fe3,Cu1,Cu2,Al1,Al2
used,400.0,0.0,39.776302,0.0,2.761272,57.462426,0.0


production costs: 98121.64
tons produced: 500.0


,grade in final product
C,2.0
Cu,0.6
Mn,1.2
